In [113]:
import pandas as pd
df1= pd.read_csv('/content/03_Item_Information_Data.csv')
df2= pd.read_csv('/content/03_Shelf_Information_Data.csv')
df3=pd.read_csv('/content/03_Customer_Behavior_Data.csv')


In [33]:
result_rows = df1[(df1['Shelf ID'] == 2) & (df1['Item ID'] == 8)]

# Hiển thị kết quả
print(result_rows)

    Shelf ID  Item ID         Name   Origin Location  Weight (g)     Price  \
39         2        8  Sữa ông thọ  Vietnam     Thấp       380.0  31,500 ₫   

           NSX         HSD  Marketing strategy  Discount   Rate  
39  2024-06-03  2025-04-29               False          0     3  


In [37]:
#1
merged_df = pd.merge(df3, df1, on=['Shelf ID', 'Item ID'], how='left')

merged_df['Time'] = merged_df['Looking at item (s)'] + merged_df['Holding the item (s)']

total_time_for_looking_and_holding = merged_df.groupby('Name')['Time'].sum().reset_index()

top_5_item_longest = total_time_for_looking_and_holding.sort_values(by='Time', ascending=False).head(5)
top_5_item_shortest = total_time_for_looking_and_holding.sort_values(by='Time', ascending=True).head(5)

print("Top 5 items got longest time for looking and holding\n",top_5_item_longest)
print("Top 5 items got shortest time for looking and holding\n",top_5_item_shortest)

Top 5 items got longest time for looking and holding
                      Name   Time
108   Sữa chua uống Probi  22896
109  Sữa chua uống Yakult  22896
110           Sữa ông thọ  13939
4           Bim bim Oishi  13866
77   Snack khoai tây Lays  13362
Top 5 items got shortest time for looking and holding
                                                   Name  Time
93                                       Sách Ðạo Tình  1152
113                                    Truyện Doraemon  1378
90                            Sách cho trẻ từ 1-2 tuổi  1421
87                          Sách Thám tử Sherlock Home  1463
116  Tã dán sơ sinh Bobby siêu mỏng Newborn 70 miế...  1669


In [5]:
#2
merged_df = pd.merge(df3, df1, on=['Shelf ID', 'Item ID'], how='left')
merged_df['Picking up item'] = merged_df['Picking up item'].astype(int)
merged_df['Returning item'] = merged_df['Returning item'].astype(int)
merged_df['Pick and return']=(merged_df['Picking up item']==1) & (merged_df['Returning item']==1)


item_counts=merged_df[merged_df['Pick and return']].groupby('Name').size().reset_index(name='Time pick and return')
top_5_item=item_counts.sort_values(by='Time pick and return',ascending=False).head(5)
print ("Top 5 pick and return\n",top_5_item)

Top 5 pick and return
                              Name  Time pick and return
0   4 hộp sữa lúa mạch Milo 180ml                   134
76           Snack khoai tây Lays                   127
50                   Mý ý SG Food                   117
59                Nước lẩu Barona                   116
6                   Bánh Chocopie                   114


In [65]:
#3
def classify_age_group(age):
    if 18 <= age <= 30:
        return 'Teenager'
    elif 31 <= age <= 60:
        return 'Middle-age'
    else:
        return 'Old'

merged_df['Age group'] = merged_df['Age'].apply(classify_age_group)


filtered_df = merged_df[(merged_df['Looking at item (s)'] > 0) &
                        (merged_df['Picking up item'] == 1) &
                        (merged_df['Holding the item (s)'] > 0) &
                        (merged_df['Putting item into bag'] == True)]
group_counts = merged_df.groupby(['Name', 'Age group']).size().reset_index(name='Number of purchases')
#Middle-age
df_loc = group_counts[group_counts['Age group'] == 'Middle-age']
max_b_row = df_loc.loc[df_loc['Number of purchases'].idxmax()]
print('Middle age\n',max_b_row)
#Teenager
df_loc = group_counts[group_counts['Age group'] == 'Teenager']
max_b_row = df_loc.loc[df_loc['Number of purchases'].idxmax()]
print('Teenager\n',max_b_row)
#Old
df_loc = group_counts[group_counts['Age group'] == 'Old']
max_b_row = df_loc.loc[df_loc['Number of purchases'].idxmax()]
print('Old\n',max_b_row)

Middle age
 Name                   Snack khoai tây Lays
Age group                        Middle-age
Number of purchases                     125
Name: 231, dtype: object
Teenager
 Name                   Sữa bột Anmum Materna
Age group                           Teenager
Number of purchases                       64
Name: 302, dtype: object
Old
 Name                   Bánh trứng Custard
Age group                             Old
Number of purchases                    70
Name: 31, dtype: object


In [114]:
#4
#Chuyển đổi 'Timestamp' sang định dạng ngày tháng
df3['Date'] = pd.to_datetime(df3['Timestamp'], unit='s')

# Lấy ngày trong tuần từ cột 'Date'
df3['Day of Week'] = df3['Date'].dt.day_name()

# Chuyển đổi cột 'Price' trong df2 sang kiểu số
df1['Price'] = df1['Price'].str.replace(r'[^\d]', '', regex=True).astype(int)

# Gộp thông tin giá vào df1
merged_df = pd.merge(df3, df1, on=['Shelf ID', 'Item ID'], how='left')
filtered_df = merged_df[(merged_df['Looking at item (s)'] > 0) &
                        (merged_df['Picking up item'] == 1) &
                        (merged_df['Holding the item (s)'] > 0) &
                        (merged_df['Putting item into bag'] == True) ]
print(filtered_df[['Price','Name']])
daily_sales = filtered_df.groupby('Day of Week')['Price'].sum().reset_index()
daily_sales = daily_sales.sort_values(by='Price', ascending=False).head(5)
print(daily_sales)




        Price                 Name
1       64500      Dầu ăn Tường An
2       79000        Dầu ăn Simply
5       64500      Dầu ăn Tường An
7      175000         Kem vinamilk
9       33000        Pizza kitkool
...       ...                  ...
15384  410000        Sửa bột Anele
15385   36000  Sữa chua uống Probi
15389  410000        Sửa bột Anele
15390    4400           Mỳ Hảo Hảo
15393    9000            Phở vifon

[8027 rows x 2 columns]
  Day of Week      Price
2    Saturday  297675600
3      Sunday  285325000
5     Tuesday   96602300
4    Thursday   88083000
0      Friday   80778300


In [ ]:
# Chuyển đổi cột 'Price' trong df2 sang kiểu số
df1['Price'] = df1['Price'].replace('₫', '', regex=True).astype(str)
df1['Price'] = df1['Price'].str.replace('.', '', regex=True).str.strip()

In [ ]:

# Tính tổng doanh thu theo ngày trong tuần
daily_revenue = merged_df.groupby('Day of Week')['Price'].sum().reset_index()
highest_revenue_day = daily_revenue.loc[daily_revenue['Giá'].idxmax()]

print(highest_revenue_day)


In [8]:
#5
# Đếm số lượng người đi cho mỗi nhóm tuổi
age_group_counts = merged_df['Age group'].value_counts().reset_index()
age_group_counts.columns = ['Age group', 'Number of purchases']

# Nhóm tuổi có lượng người đi nhiều nhất
most_number_of_purchases_age_group = age_group_counts.iloc[0]

print(age_group_counts)
print("\nThe age group has the largest number of people going:")
print(most_number_of_purchases_age_group)

    Age group  Number of purchases
0  Middle-age                 7617
1         Old                 4203
2    Teenager                 3574

The age group has the largest number of people going:
Age group              Middle-age
Number of purchases          7617
Name: 0, dtype: object


In [12]:
6#
merged_df = pd.merge(df3, df1, on=['Shelf ID', 'Item ID'], how='left') #df3:customer_behavior, df1: item_info

merged_df['Picking up item'] = merged_df['Picking up item'].astype(int)
# Lọc dữ liệu: Thời gian đi và về > 0 và Đi = True
filtered_df = merged_df[(merged_df['Looking at item (s)'] > 0) &
                        (merged_df['Picking up item'] == 1) &
                        (merged_df['Holding the item (s)'] > 0) &
                        (merged_df['Putting item into bag'] == True) &
                        (merged_df['Discount ']>0)]

# Nhóm theo 'Tên địa điểm' và tính số lượt chốt
item_summary = filtered_df.groupby('Name').size().reset_index(name='Number of purchases of discount item')

# Sắp xếp theo số lượt chốt giảm dần và lấy top 5
top_5_item = item_summary.sort_values(by='Number of purchases of discount item', ascending=False).head(5)

print("Top 5 sản phẩm giảm giá được người dùng chốt nhiều nhất:")
print(top_5_item)

Top 5 sản phẩm giảm giá được người dùng chốt nhiều nhất:
                  Name  Number of purchases of discount item
4   Bánh trứng Custard                                   188
22      Kem tràng tiền                                   180
49        Sữa bột Milo                                   179
13      Dầu gội Romano                                   113
23      Khăn mặt Shine                                   112


In [14]:
#7
merged_df = pd.merge(df3, df1, on=['Shelf ID', 'Item ID'], how='left') #df3:customer_behavior, df1: item_info

merged_df['Picking up item'] = merged_df['Picking up item'].astype(int)
filtered_df = merged_df[(merged_df['Looking at item (s)'] > 0) &
                        (merged_df['Picking up item'] == 1) &
                        (merged_df['Holding the item (s)'] > 0) &
                        (merged_df['Putting item into bag'] == True) &
                        (merged_df['Marketing strategy'] == True)]

# Nhóm theo 'Tên địa điểm' và tính số lượt chốt
item_summary = filtered_df.groupby('Name').size().reset_index(name='Number of purchases of marketed item')

# Sắp xếp theo số lượt chốt giảm dần và lấy top 5
top_5_item = item_summary.sort_values(by='Number of purchases of marketed item', ascending=False).head(5)

print("Top 5 nhãn hàng được quảng cáo được người dùng chốt nhiều nhất:")
print(top_5_item)


Top 5 nhãn hàng được quảng cáo được người dùng chốt nhiều nhất:
                  Name  Number of purchases of marketed item
0   Bánh trứng Custard                                   188
6       Kem tràng tiền                                   180
14        Sữa bột Milo                                   179
7       Khăn mặt Shine                                   112
8       Khăn tắm Shine                                   106


In [16]:
#8
merged_df = pd.merge(df3, df2, on=['Shelf ID'], how='left')
merged_df['Total_time'] = merged_df['Looking at item (s)'] + merged_df['Holding the item (s)']
total_of_time_concern = merged_df.groupby('Description')['Total_time'].sum().reset_index()

print("Time of concern \n",total_of_time_concern)

merged_df['Picking up item'] = merged_df['Picking up item'].astype(int).fillna(0)
merged_df['Returning item'] = merged_df['Returning item'].astype(int).fillna(0)
merged_df['Putting item into bag'] = merged_df['Putting item into bag'].fillna(False).astype(int)
merged_df['Taking item out of bag'] = merged_df['Taking item out of bag'].fillna(False).astype(int)
merged_df['Putting item into bag in the 2nd time'] = merged_df['Putting item into bag in the 2nd time'].fillna(False).astype(int)
merged_df['Number of interactions'] = merged_df['Picking up item'] + merged_df['Returning item'] + merged_df['Putting item into bag'] + merged_df['Taking item out of bag'] + merged_df['Putting item into bag in the 2nd time']
total_of_interactions = merged_df.groupby('Description')['Number of interactions'].sum().reset_index()

print("\n Total of interactions\n",total_of_interactions)
top_5_list_df=pd.merge(total_of_time_concern,total_of_interactions, on=['Description'], how='left')
top_5_list_df['Concern over interaction'] = top_5_list_df['Total_time'] / top_5_list_df['Number of interactions']

# Sắp xếp DataFrame theo cột 'C' theo thứ tự giảm dần và lấy 5 giá trị cao nhất
top_5_values = top_5_list_df.sort_values(by='Concern over interaction', ascending=False).head(5)
print('\nTop 3 shelfs got most Concern over interaction\n ',top_5_values)


Time of concern 
             Description  Total_time
0         Quầy bánh kẹo      153458
1         Quầy gia dụng       47380
2           Quầy gia vị      108934
3      Quầy hoá mỹ phẩm      120088
4  Quầy sách và đồ chơi       42914
5              Quầy sữa      106461
6        Quầy thực phẩm       77919
7        Quầy đông lạnh      201638

 Total of interactions
             Description  Number of interactions
0         Quầy bánh kẹo                    7668
1         Quầy gia dụng                    2014
2           Quầy gia vị                    5163
3      Quầy hoá mỹ phẩm                    6522
4  Quầy sách và đồ chơi                    3000
5              Quầy sữa                    5271
6        Quầy thực phẩm                    3584
7        Quầy đông lạnh                    9211

Top 3 shelfs got most Concern over interaction
        Description  Total_time  Number of interactions  \
1   Quầy gia dụng       47380                    2014   
7  Quầy đông lạnh      201638        

In [118]:
#9
merged_df = df3.merge(df2, on=['Shelf ID'], how='left')

merged_df['Picking up item'] = merged_df['Picking up item'].astype(int)
# Lọc dữ liệu: Thời gian đi và về > 0 và Đi = True
filtered_df = merged_df[(merged_df['Looking at item (s)'] > 0) &
                        (merged_df['Picking up item'] == 1) &
                        (merged_df['Holding the item (s)'] > 0) &
                        (merged_df['Putting item into bag'] == True) ]

item_summary = filtered_df.groupby('Shelf ID').size().reset_index(name='Number of purchases of each shelf') #Note: chưa in ra tên quầy( cần bổ sung )
top_5_item_of_shelf = item_summary.sort_values(by='Number of purchases of each shelf', ascending=False).head()

print("Top 5 Quầy được người dùng chốt nhiều nhất:")

print(top_5_item_of_shelf)


Top 5 Quầy được người dùng chốt nhiều nhất:
   Shelf ID  Number of purchases of each shelf
7         7                               1607
1         1                               1387
0         0                               1354
2         2                               1026
4         4                                962


In [31]:
#10
merged_df = df3.merge(df2, on=['Shelf ID'], how='left')
shelf_x_by_customer = merged_df.groupby('Person ID')['Shelf ID'].unique().reset_index()
# Lặp qua từng hàng trong DataFrame và truy xuất tất cả các mảng NumPy trong cột 'Shelf ID'
all_shelf_ids = [item for sublist in shelf_x_by_customer['Shelf ID'] for item in sublist]

# Bước 2: Tạo các cặp giá trị liên tiếp
pairs = [(all_shelf_ids[i], all_shelf_ids[i+1]) for i in range(len(all_shelf_ids) - 1)]

# Bước 3: Đếm tần suất xuất hiện của các cặp
pair_counts = Counter(pairs)

# Chuyển đổi kết quả thành DataFrame
pair_df = pd.DataFrame(pair_counts.items(), columns=['Pair', 'Count'])

# Bước 4: Sắp xếp và tìm các cặp phổ biến nhất
top_pairs = pair_df.sort_values(by='Count', ascending=False).head(5)

# Hiển thị kết quả
print(top_pairs)


      Pair  Count
29  (7, 0)    153
5   (0, 7)    144
12  (1, 7)    139
11  (7, 1)    124
13  (7, 4)    116


Những sản phẩm có xu hướng đi với nhau

In [120]:
merged_df = df3.merge(df1, on=['Shelf ID'], how='left')
shelf_x_by_customer = merged_df.groupby('Person ID')['Name'].unique().reset_index()
# Lặp qua từng hàng trong DataFrame và truy xuất tất cả các mảng NumPy trong cột 'Shelf ID'
all_shelf_ids = [item for sublist in shelf_x_by_customer['Name'] for item in sublist]

# Bước 2: Tạo các cặp giá trị liên tiếp
pairs = [(all_shelf_ids[i], all_shelf_ids[i+1]) for i in range(len(all_shelf_ids) - 1)]

# Bước 3: Đếm tần suất xuất hiện của các cặp
pair_counts = Counter(pairs)

# Chuyển đổi kết quả thành DataFrame
pair_df = pd.DataFrame(pair_counts.items(), columns=['Pair', 'Count'])

# Bước 4: Sắp xếp và tìm các cặp phổ biến nhất
top_pairs = pair_df.sort_values(by='Count', ascending=False).head()

# Hiển thị kết quả
print(top_pairs)


                                      Pair  Count
26    (Sữa chua uống Probi, Sữa chua Susu)    718
31         (Mý ý SG Food, Thanh cua Akira)    718
24  (Sữa chua Ba vì, Sữa chua uống Yakult)    718
23     (Sữa chua Mộc Châu, Sữa chua Ba vì)    718
22        (Sửa chua TH, Sữa chua Mộc Châu)    718


Top sản phẩm mua ít nhất và mua nhiều nhất

In [116]:
merged_df = pd.merge(df3, df1, on=['Shelf ID', 'Item ID'], how='left') #df3:customer_behavior, df1: item_info

merged_df['Picking up item'] = merged_df['Picking up item'].astype(int)
filtered_df = merged_df[(merged_df['Looking at item (s)'] > 0) &
                        (merged_df['Picking up item'] == 1) &
                        (merged_df['Holding the item (s)'] > 0) &
                        (merged_df['Putting item into bag'] == True)]

# Nhóm theo 'Tên địa điểm' và tính số lượt chốt
item_summary = filtered_df.groupby('Name').size().reset_index(name='Number of purchases of marketed item')

# Sắp xếp theo số lượt chốt giảm dần và lấy top 5
top_5_item_nhieu = item_summary.sort_values(by='Number of purchases of marketed item', ascending=False).head(5)
top_5_item_it = item_summary.sort_values(by='Number of purchases of marketed item', ascending=True).head(5)
print("Top 5 nhãn hàng được được người dùng mua nhiều nhất:")
print(top_5_item_nhieu)
print("\nTop 5 nhãn hàng được được người dùng mua ít nhất:\n",top_5_item_it)


Top 5 nhãn hàng được được người dùng mua nhiều nhất:
                                           Name  \
46     Lốc 4 hộp sữa tươi ít đường TH true MILK   
10                           Bánh trứng Custard   
36                               Kem tràng tiền   
103                                Sữa bột Milo   
115  Tã dán Merries size NB 76 miếng (dưới 5kg)   

     Number of purchases of marketed item  
46                                    221  
10                                    188  
36                                    180  
103                                   179  
115                                   116  

Top 5 nhãn hàng được được người dùng mua ít nhất:
                                       Name  \
93                           Sách Ðạo Tình   
132                                 Ðậu đỏ   
59                              Nui Safoco   
89   Sách cho trẻ sơ sinh đến 6 tháng tuổi   
53                              Mỳ Hảo Hảo   

     Number of purchases of marketed item  
93 

Quầy được người dùng mua ít nhất và nhiều nhất

In [119]:
merged_df = df3.merge(df2, on=['Shelf ID'], how='left')

merged_df['Picking up item'] = merged_df['Picking up item'].astype(int)
# Lọc dữ liệu: Thời gian đi và về > 0 và Đi = True
filtered_df = merged_df[(merged_df['Looking at item (s)'] > 0) &
                        (merged_df['Picking up item'] == 1) &
                        (merged_df['Holding the item (s)'] > 0) &
                        (merged_df['Putting item into bag'] == True) ]

item_summary = filtered_df.groupby('Shelf ID').size().reset_index(name='Number of purchases of each shelf') #Note: chưa in ra tên quầy( cần bổ sung )
top_5_item_of_shelf = item_summary.sort_values(by='Number of purchases of each shelf', ascending=False).head(5)
top_5_item_of_shelf_it = item_summary.sort_values(by='Number of purchases of each shelf', ascending=True).head(5)

print("Top 5 Quầy được người dùng chốt nhiều nhất:")

print(top_5_item_of_shelf)
print("Top 5 Quầy được người dùng chốt ít nhất:")
print(top_5_item_of_shelf_it)


Top 5 Quầy được người dùng chốt nhiều nhất:
   Shelf ID  Number of purchases of each shelf
7         7                               1607
1         1                               1387
0         0                               1354
2         2                               1026
4         4                                962
Top 5 Quầy được người dùng chốt ít nhất:
   Shelf ID  Number of purchases of each shelf
6         6                                474
5         5                                516
3         3                                701
4         4                                962
2         2                               1026
